Data imputation

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/data_clean.csv', sep=',', encoding='latin-1')
print(df.head())
print(df.columns)
print(df.shape)


    SEQN  RIDAGEYR  RIAGENDR  RIDRETH1  RIDRETH3  DMDCITZN  DMDEDUC2  MIALANG  \
0  73557        69         1         4         4       1.0       3.0      1.0   
1  73558        54         1         3         3       1.0       3.0      1.0   
2  73559        72         1         3         3       1.0       4.0      1.0   
3  73561        73         2         3         3       1.0       5.0      1.0   
4  73562        56         1         1         1       1.0       4.0      1.0   

   DMDHHSIZ  INDHHIN2  ...  PAQ635  PAQ650  PAQ665  PAD680  PAQ706  PAQ710  \
0         3       4.0  ...       2       2       2   600.0     NaN     2.0   
1         4       7.0  ...       2       2       2   540.0     NaN     4.0   
2         2      10.0  ...       2       2       1   300.0     NaN     4.0   
3         2      15.0  ...       2       2       2   480.0     NaN     1.0   
4         1       9.0  ...       2       2       2   360.0     NaN     5.0   

   LBXTC  highLDL  highbp        bmi  
0  16

Data imputation for 3 columns

In [2]:
df = df[["BPXPLS", "BMXARMC", "bmi", "highbp" ]]
# df["highbp"].isnull().sum()
# df["highbp"]


Convert dataframe to numpy array

In [3]:
# df[df.isnull()] = 0
df.shape
np_array = df.to_numpy(copy=True, na_value=np.nan)

# for column in df.columns:
#     df_null_inx.append(df[df[column].isnull()].index.tolist())

# # df_null_inx

# for column in df.columns:
#     df[column].fillna((df[column].mean()), inplace=True)


To test our code, we will first delete values ourselves and compare output

In [20]:
data1 = np_array[~np.isnan(np_array).any(axis=1), :] #delete rows with any missing values
#print(np.where(np.isnan(data1)))

#create nans at random
missing_indices = np.random.choice(len(data1.flatten()), size = int(len(data1.flatten())/10)) 
#randomly select values from length ie flat array
print(missing_indices)

data1_test = data1
data1_test.flat[missing_indices] = np.nan
print(data1_test)

## data1 is our ground truth, data1_test is test set

[22128 20725 20047 ...  2724  5226 14841]
[[        nan 35.3        26.68376063  0.        ]
 [74.         34.7        28.6324502   1.        ]
 [68.         33.5        28.92930024  1.        ]
 ...
 [72.58734271 31.         26.79702004  0.36274702]
 [60.         29.9        24.46863363  0.        ]
 [80.                 nan 34.01503875  1.        ]]


Get non-missing indices

In [21]:

## FIX: delete this line when no longer working with data1_test
np_array = data1_test


# nonmissing_indices = np.flatnonzero(np_array > 0.1)
# nonmissing_indices = np_array.flat[np.argwhere(np.isfinite(np_array))]
# nonmissing_indices = np_array.flat[np.isfinite(np_array.flat)]
# nonmissing_indices = np.flatnonzero(~ np.isnan(np_array))
nonmissing_indices = np.argwhere(~ np.isnan(np_array.flatten())).reshape(-1)
missing_indices = np.argwhere(np.isnan(np_array.flatten())).reshape(-1)
# nonmissing_indices.reshape(-1)
print(nonmissing_indices)
print(len(nonmissing_indices))
print(len(missing_indices))

# u, s, v = np.linalg.svd(df)


[    1     2     3 ... 23072 23074 23075]
20871
2205


Split into training, and test

fill an array with available data

In [22]:
rating_matrix_ini = np.zeros(np_array.shape)
rating_matrix_ini.flat[nonmissing_indices] = np_array.flat[nonmissing_indices]


# rating_matrix_ini
# for column in np_array.shape[1]:
#     np_array[:,column] = 

Impute missing data

In [23]:
np.isnan(np_array)

array([[ True, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       ...,
       [False, False, False, False],
       [False, False, False, False],
       [False,  True, False, False]])

In [24]:
# rating_matrix_ini[6,:]
# for column in range(np_array.shape[1]):
#     np_array[np.where(np_array[:,column] == np.nan),column] = np.mean(np_array[:,column])
col_mean = np.nanmean(np_array, axis=0)
inds = np.where(np.isnan(np_array))
np_array[inds] = np.take(col_mean, inds[1])




In [25]:
np_array.shape
np.isnan(np_array)

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       ...,
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False]])

Fit low rank model for rank 1

In [34]:
def fit_low_rank_model(rank,rating_matrix_ini,train_ind,train_data,n_iter,convergence_thresh,verbose, data1, missing_indices):
    """Fit the low rank model. 
    Return the estimation of the low rank model - (n_movies * n_users) matrix

    Keyword arguments:
    rank -- the rank of low rank model
    rating_matrix_ini -- imputed initialization
    train_ind -- index of training data
    train_data -- ratings of training set
    n_iter -- the max number of iterations
    convergence_thresh -- the threshold of convergence to 0
    """
    previous_fitting_error = 100
    # Initialization
    low_rank_estimate = np.zeros(rating_matrix_ini.shape)
    # fill input data
    low_rank_estimate.flat[train_ind] = train_data
    # get the indexes of missing data
    missing_inds = np.where(low_rank_estimate.flat == 0)
    # fill missing data with imputed values
    low_rank_estimate.flat[missing_inds] = rating_matrix_ini.flat[missing_inds]

    
    for ind in range(n_iter):
        # Updates
        low_rank_estimate.flat[train_ind] = train_data
        u, s, v = np.linalg.svd(low_rank_estimate)
        s_matrix = s  * np.eye(len(s))
        low_rank_estimate = np.matmul(np.matmul(u[:,0:rank], s_matrix[0:rank,0:rank]), v[0:rank,:])
        # Compute error
        fitting_error = np.sqrt(((train_data - low_rank_estimate.flat[train_ind])**2).mean())
        #true fitting error, compared to true values
        true_fitting_error = np.sqrt(((data1.flat[missing_indices] - low_rank_estimate.flat[missing_indices])**2).mean())
        if verbose:
            print("Iteration " + str(ind) + " Error: " + str(fitting_error))
            print("Iteration " + str(ind) + "True Error: " + str(true_fitting_error))
        
        # Stopping criterion
        if (fitting_error <= convergence_thresh):
            print('converged, breaking')
            break
    return low_rank_estimate

n_iter = 5
convergence_thresh = 1e-4
verbose = True
rank = 1
train_data = np_array.flat[nonmissing_indices] 
estimate =fit_low_rank_model(rank,rating_matrix_ini,nonmissing_indices,
        train_data,n_iter,convergence_thresh,verbose, data1, missing_indices)
print(estimate)

Iteration 0 Error: 7.468829130259888
Iteration 0True Error: 29.561980869486945
Iteration 1 Error: 5.98527519969878
Iteration 1True Error: 22.831784347284753
Iteration 2 Error: 5.3372011567625615
Iteration 2True Error: 18.31956413970668
Iteration 3 Error: 4.96966001303373
Iteration 3True Error: 15.247428706950522
Iteration 4 Error: 4.754995805395935
Iteration 4True Error: 13.176518639463726
[[56.97595451 26.31591662 23.32246418  0.29052326]
 [73.68492259 34.03341454 30.16209175  0.37572313]
 [69.02660514 31.8818421  28.25526206  0.35197013]
 ...
 [70.87796769 32.73694498 29.01309643  0.36141032]
 [60.70329694 28.03749256 24.84820975  0.30952916]
 [80.48338849 37.1734736  32.94496707  0.41038885]]


Compare data1_test post imputation to data1 (true value)

In [27]:
np.linalg.norm(data1 - estimate, ord = "fro")

924.5142065093784

In [29]:
np.linalg.norm(np_array - estimate, ord = "fro")

924.5142065093784

In [32]:
test_data = np_array.flat[missing_indices]
np.sqrt(((test_data - estimate.flat[missing_indices])**2).mean())

13.176518639463726